In [5]:
import os, uuid
from flask import Flask, request, render_template, jsonify
from processors import run_analytics_with_speed

BASE = os.getcwd()
app = Flask(__name__, template_folder=os.path.join(BASE,"templates"),
                     static_folder=os.path.join(BASE,"static"))

OUT_DIR = os.path.join(BASE, "static", "outputs")
os.makedirs(OUT_DIR, exist_ok=True)

@app.route("/", methods=["GET"])
def home():
    return render_template("index.html")

@app.route("/analyze", methods=["POST"])
def analyze():
    if "video" not in request.files or request.files["video"].filename == "":
        return jsonify({"error":"حمّل فيديو بالحقل 'video'"}), 400

    f = request.files["video"]
    vid_id = uuid.uuid4().hex
    in_path  = os.path.join(OUT_DIR, f"in_{vid_id}.mp4")
    out_path = os.path.join(OUT_DIR, f"out_{vid_id}.mp4")
    csv_path = os.path.join(OUT_DIR, f"stats_{vid_id}.csv")
    f.save(in_path)

    # بارامترات
    weights = request.form.get("weights","yolov8s.pt")
    conf = float(request.form.get("conf","0.15"))
    iou  = float(request.form.get("iou","0.45"))
    imgsz = int(request.form.get("imgsz","960"))
    speed_limit_kmh = float(request.form.get("speed_limit_kmh","60"))

    # ppm / calibration
    ppm = request.form.get("ppm","").strip()
    pixels_per_meter = float(ppm) if ppm else None

    cal_x1 = request.form.get("cal_x1","").strip()
    cal_y1 = request.form.get("cal_y1","").strip()
    cal_x2 = request.form.get("cal_x2","").strip()
    cal_y2 = request.form.get("cal_y2","").strip()
    cal_real_m = request.form.get("cal_real_m","").strip()

    calibration_points = None
    calibration_real_m = None
    if (cal_x1 and cal_y1 and cal_x2 and cal_y2 and cal_real_m):
        calibration_points = ((float(cal_x1), float(cal_y1)), (float(cal_x2), float(cal_y2)))
        calibration_real_m = float(cal_real_m)

    if not pixels_per_meter and not calibration_points:
        pixels_per_meter = 100.0  # fallback

    summary = run_analytics_with_speed(
        input_path=in_path,
        output_video_path=out_path,
        csv_path=csv_path,
        weights=weights, conf=conf, iou=iou, imgsz=imgsz,
        speed_limit_kmh=speed_limit_kmh,
        pixels_per_meter=pixels_per_meter,
        calibration_points=calibration_points,
        calibration_real_m=calibration_real_m
    )

    return jsonify({
        "summary": summary,
        "video_url": f"/static/outputs/{os.path.basename(out_path)}",
        "csv_url": f"/static/outputs/{os.path.basename(csv_path)}"
    })

if __name__ == "__main__":
    port = int(os.environ.get("PORT","5056"))
    print(f"Running on http://localhost:{port}")
    app.run(host="0.0.0.0", port=port, debug=False)


Running on http://localhost:5056
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5056
 * Running on http://192.168.25.123:5056
Press CTRL+C to quit
127.0.0.1 - - [10/Nov/2025 16:39:36] "GET / HTTP/1.1" 200 -



video 1/1 (frame 1/335) /Users/abd/Desktop/OFPPT/smart_speed_radar/static/outputs/in_20149dd3b4c94d9592cdc862b14b9a3a.mp4: 544x960 8 cars, 113.3ms
video 1/1 (frame 2/335) /Users/abd/Desktop/OFPPT/smart_speed_radar/static/outputs/in_20149dd3b4c94d9592cdc862b14b9a3a.mp4: 544x960 7 cars, 112.5ms
video 1/1 (frame 3/335) /Users/abd/Desktop/OFPPT/smart_speed_radar/static/outputs/in_20149dd3b4c94d9592cdc862b14b9a3a.mp4: 544x960 7 cars, 116.7ms
video 1/1 (frame 4/335) /Users/abd/Desktop/OFPPT/smart_speed_radar/static/outputs/in_20149dd3b4c94d9592cdc862b14b9a3a.mp4: 544x960 8 cars, 110.0ms
video 1/1 (frame 5/335) /Users/abd/Desktop/OFPPT/smart_speed_radar/static/outputs/in_20149dd3b4c94d9592cdc862b14b9a3a.mp4: 544x960 8 cars, 109.4ms
video 1/1 (frame 6/335) /Users/abd/Desktop/OFPPT/smart_speed_radar/static/outputs/in_20149dd3b4c94d9592cdc862b14b9a3a.mp4: 544x960 8 cars, 108.4ms
video 1/1 (frame 7/335) /Users/abd/Desktop/OFPPT/smart_speed_radar/static/outputs/in_20149dd3b4c94d9592cdc862b14b9a3a

127.0.0.1 - - [10/Nov/2025 16:40:30] "POST /analyze HTTP/1.1" 200 -
127.0.0.1 - - [10/Nov/2025 16:40:30] "GET /static/outputs/out_20149dd3b4c94d9592cdc862b14b9a3a.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [10/Nov/2025 16:40:30] "GET /static/outputs/out_20149dd3b4c94d9592cdc862b14b9a3a.mp4 HTTP/1.1" 206 -


In [1]:
import os, uuid, sqlite3
from pathlib import Path
from werkzeug.utils import secure_filename
from werkzeug.security import generate_password_hash, check_password_hash
from flask import Flask, request, render_template, jsonify, url_for, redirect, session

from processors import run_analytics_with_speed

# ---------------- إعداد التطبيق ----------------
BASE = Path.cwd()
app = Flask(__name__, static_folder="static", template_folder="templates")
app.secret_key = "secret_key_dyalk"

OUT_DIR = BASE / "static" / "outputs"
OUT_DIR.mkdir(parents=True, exist_ok=True)

ALLOWED_EXTS = {".mp4", ".mov", ".mkv", ".avi", ".webm"}
ALLOWED_MIMES = {
    "video/mp4", "video/quicktime", "video/x-matroska",
    "video/x-msvideo", "video/webm"
}

# ---------------- قاعدة البيانات ----------------
def get_db_connection():
    conn = sqlite3.connect("users.db")
    conn.row_factory = sqlite3.Row
    return conn

def init_db():
    conn = get_db_connection()
    c = conn.cursor()
    c.execute("""
        CREATE TABLE IF NOT EXISTS users (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            username TEXT UNIQUE NOT NULL,
            password TEXT NOT NULL
        )
    """)
    username = "admin"
    password = generate_password_hash("123456")
    c.execute("INSERT OR IGNORE INTO users (username, password) VALUES (?, ?)", (username, password))
    conn.commit()
    conn.close()

init_db()

# ---------------- Routes ----------------

# تسجيل الدخول
@app.route("/login", methods=["GET", "POST"])
def login():
    if request.method == "POST":
        username = request.form["username"].strip()
        password = request.form["password"].strip()
        conn = get_db_connection()
        user = conn.execute("SELECT * FROM users WHERE username = ?", (username,)).fetchone()
        conn.close()
        if user and check_password_hash(user["password"], password):
            session["user"] = username
            return redirect(url_for("home"))
        else:
            return render_template("login.html", error="❌ اسم المستخدم أو كلمة المرور غير صحيحة")
    return render_template("login.html")


# إنشاء حساب جديد
@app.route("/register", methods=["GET", "POST"])
def register():
    if request.method == "POST":
        username = request.form["username"].strip()
        password = request.form["password"].strip()
        confirm = request.form["confirm"].strip()
        if password != confirm:
            return render_template("register.html", error="❌ كلمتا المرور غير متطابقتين")
        conn = get_db_connection()
        user = conn.execute("SELECT * FROM users WHERE username = ?", (username,)).fetchone()
        if user:
            conn.close()
            return render_template("register.html", error="⚠️ اسم المستخدم مستعمل بالفعل")
        hashed_pw = generate_password_hash(password)
        conn.execute("INSERT INTO users (username, password) VALUES (?, ?)", (username, hashed_pw))
        conn.commit()
        conn.close()
        return redirect(url_for("login"))
    return render_template("register.html")


# تسجيل الخروج
@app.route("/logout")
def logout():
    session.pop("user", None)
    return redirect(url_for("login"))


# الصفحة الرئيسية (محمية)
@app.route("/", methods=["GET"])
def home():
    if "user" not in session:
        return redirect(url_for("login"))
    return render_template("index.html", user=session["user"])


# لوحة تحكم المشرف
@app.route("/admin")
def admin_dashboard():
    if "user" not in session:
        return redirect(url_for("login"))
    if session["user"] != "admin":
        return "🚫 الوصول مرفوض: خاصك تكون admin", 403
    conn = get_db_connection()
    users = conn.execute("SELECT id, username FROM users ORDER BY id ASC").fetchall()
    conn.close()
    return render_template("admin.html", users=users)


# حذف مستخدم
@app.route("/delete_user/<int:user_id>")
def delete_user(user_id):
    if "user" not in session or session["user"] != "admin":
        return redirect(url_for("login"))
    conn = get_db_connection()
    conn.execute("DELETE FROM users WHERE id = ?", (user_id,))
    conn.commit()
    conn.close()
    return redirect(url_for("admin_dashboard"))


# تصدير قاعدة البيانات إلى data.sql
@app.route("/export_db")
def export_db():
    if "user" not in session or session["user"] != "admin":
        return "🚫 غير مصرح لك", 403
    conn = get_db_connection()
    data = []
    for row in conn.execute("SELECT * FROM users").fetchall():
        data.append(f"INSERT INTO users (id, username, password) VALUES ({row['id']}, '{row['username']}', '{row['password']}');")
    conn.close()
    export_path = Path("data.sql")
    with open(export_path, "w", encoding="utf-8") as f:
        f.write("-- Backup of users.db\n")
        f.write("\n".join(data))
    return f"✅ تم إنشاء ملف {export_path.absolute()}"


# تحليل الفيديوهات - بنفس طريقة الكود اللي كيشغل الفيديو
@app.route("/analyze", methods=["POST"])
def analyze():
    if "user" not in session:
        return jsonify({"error": "الرجاء تسجيل الدخول أولا"}), 401

    if "video" not in request.files or request.files["video"].filename == "":
        return jsonify({"error":"حمّل فيديو بالحقل 'video'"}), 400

    f = request.files["video"]
    vid_id = uuid.uuid4().hex
    in_path  = OUT_DIR / f"in_{vid_id}.mp4"
    out_path = OUT_DIR / f"out_{vid_id}.mp4"
    csv_path = OUT_DIR / f"stats_{vid_id}.csv"
    f.save(in_path)

    weights = request.form.get("weights", "yolov8s.pt")
    conf = float(request.form.get("conf", "0.15"))
    iou  = float(request.form.get("iou", "0.45"))
    imgsz = int(request.form.get("imgsz", "960"))
    speed_limit_kmh = float(request.form.get("speed_limit_kmh", "60"))

    ppm = request.form.get("ppm", "").strip()
    pixels_per_meter = float(ppm) if ppm else 100.0

    summary = run_analytics_with_speed(
        input_path=str(in_path),
        output_video_path=str(out_path),
        csv_path=str(csv_path),
        weights=weights, conf=conf, iou=iou, imgsz=imgsz,
        speed_limit_kmh=speed_limit_kmh,
        pixels_per_meter=pixels_per_meter
    )

    return jsonify({
        "summary": summary,
        "video_url": f"/static/outputs/{os.path.basename(out_path)}",
        "csv_url": f"/static/outputs/{os.path.basename(csv_path)}"
    })


# ---------------- تشغيل السيرفر ----------------
if __name__ == "__main__":
    port = int(os.environ.get("PORT", "5055"))
    print(f"Running on http://localhost:{port}")
    app.run(host="0.0.0.0", port=port, debug=False)

Running on http://localhost:5055
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5055
 * Running on http://192.168.25.123:5055
Press CTRL+C to quit
127.0.0.1 - - [10/Nov/2025 17:37:01] "GET / HTTP/1.1" 302 -
127.0.0.1 - - [10/Nov/2025 17:37:01] "GET /login HTTP/1.1" 200 -


In [4]:
# ============================================
# 🔹 Imports
# ============================================
import os, uuid, sqlite3, base64
from pathlib import Path
from IPython.display import display, HTML, Video, clear_output, FileLink
import ipywidgets as widgets
import pandas as pd
from werkzeug.security import generate_password_hash, check_password_hash
from processors import run_analytics_with_speed

# ============================================
# 🔹 إعداد المشروع
# ============================================
BASE = Path.cwd()
OUT_DIR = BASE / "outputs"
OUT_DIR.mkdir(parents=True, exist_ok=True)

# قاعدة البيانات
def get_db_connection():
    conn = sqlite3.connect("users.db")
    conn.row_factory = sqlite3.Row
    return conn

def init_db():
    conn = get_db_connection()
    c = conn.cursor()
    c.execute("""
        CREATE TABLE IF NOT EXISTS users (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            username TEXT UNIQUE NOT NULL,
            password TEXT NOT NULL
        )
    """)
    username = "admin"
    password = generate_password_hash("123456")
    c.execute("INSERT OR IGNORE INTO users (username, password) VALUES (?, ?)", (username, password))
    conn.commit()
    conn.close()

init_db()

# ============================================
# 🔹 Widgets المكونات الرسومية
# ============================================
style = {'description_width': '150px'}
layout = widgets.Layout(width='400px')

login_user = widgets.Text(description="👤 Nom d'utilisateur:", style=style, layout=layout)
login_pass = widgets.Password(description="🔑 Mot de passe:", style=style, layout=layout)
login_btn = widgets.Button(description="Connexion", button_style='success')
register_btn = widgets.Button(description="Créer un compte", button_style='info')
output_box = widgets.Output()

# ============================================
# 🔹 Fonctions d'authentification
# ============================================
def authenticate_user(username, password):
    conn = get_db_connection()
    user = conn.execute("SELECT * FROM users WHERE username = ?", (username,)).fetchone()
    conn.close()
    if user and check_password_hash(user["password"], password):
        return True
    return False

def register_user(username, password):
    conn = get_db_connection()
    user = conn.execute("SELECT * FROM users WHERE username = ?", (username,)).fetchone()
    if user:
        conn.close()
        return False
    hashed_pw = generate_password_hash(password)
    conn.execute("INSERT INTO users (username, password) VALUES (?, ?)", (username, hashed_pw))
    conn.commit()
    conn.close()
    return True

# ============================================
# 🔹 واجهة تسجيل الدخول
# ============================================
def show_login():
    clear_output()
    display(HTML("<h2 style='color:#2b7; text-align:center;'>🎬 Système d’analyse vidéo intelligent</h2>"))
    display(login_user, login_pass, login_btn, register_btn, output_box)

def on_login_clicked(_):
    with output_box:
        output_box.clear_output()
        user = login_user.value.strip()
        pwd = login_pass.value.strip()
        if authenticate_user(user, pwd):
            print(f"✅ Bienvenue {user} !")
            show_dashboard(user)
        else:
            print("❌ Nom d’utilisateur ou mot de passe incorrect")

def on_register_clicked(_):
    with output_box:
        output_box.clear_output()
        user = login_user.value.strip()
        pwd = login_pass.value.strip()
        if not user or not pwd:
            print("⚠️ Entrer un nom et mot de passe valides.")
            return
        if register_user(user, pwd):
            print("✅ Compte créé avec succès. Vous pouvez vous connecter.")
        else:
            print("⚠️ Nom d’utilisateur déjà utilisé.")

login_btn.on_click(on_login_clicked)
register_btn.on_click(on_register_clicked)

# ============================================
# 🔹 واجهة التحليل
# ============================================
def show_dashboard(username):
    clear_output()
    display(HTML(f"<h3>👋 Bonjour, <b>{username}</b></h3>"))
    logout_btn = widgets.Button(description="Déconnexion", button_style='danger')
    display(logout_btn)

    def on_logout(_):
        show_login()
    logout_btn.on_click(on_logout)

    file_upload = widgets.FileUpload(
        accept='.mp4,.mov,.avi,.mkv,.webm',
        multiple=False,
        description='📁 Choisir une vidéo'
    )

    weights = widgets.Dropdown(options=['yolov8s.pt', 'yolov8m.pt', 'yolov8l.pt'], description='Modèle YOLO:')
    conf = widgets.FloatSlider(value=0.25, min=0.1, max=1.0, step=0.05, description='Confiance:')
    iou = widgets.FloatSlider(value=0.45, min=0.1, max=1.0, step=0.05, description='IOU:')
    imgsz = widgets.Dropdown(options=[640, 960, 1280], value=960, description='Taille image:')
    speed_limit = widgets.IntSlider(value=60, min=10, max=200, description='Limite km/h:')
    ppm = widgets.FloatText(value=100.0, description='Pixels/mètre:')

    analyze_btn = widgets.Button(description="🚀 Lancer l’analyse", button_style='success')
    output_video = widgets.Output()

    display(file_upload, weights, conf, iou, imgsz, speed_limit, ppm, analyze_btn, output_video)

    def on_analyze(_):
        with output_video:
            output_video.clear_output()
            if not file_upload.value:
                print("⚠️ Aucun fichier sélectionné.")
                return

            # Sauvegarde fichier
            vid_id = uuid.uuid4().hex
            in_path = OUT_DIR / f"in_{vid_id}.mp4"
            out_path = OUT_DIR / f"out_{vid_id}.mp4"
            csv_path = OUT_DIR / f"stats_{vid_id}.csv"

            # Correction ici ✅
            file_info = file_upload.value[0]
            with open(in_path, "wb") as f:
                f.write(file_info["content"])

            print("⏳ Analyse en cours...")

            try:
                summary = run_analytics_with_speed(
                    input_path=str(in_path),
                    output_video_path=str(out_path),
                    csv_path=str(csv_path),
                    weights=weights.value,
                    conf=conf.value,
                    iou=iou.value,
                    imgsz=imgsz.value,
                    speed_limit_kmh=speed_limit.value,
                    pixels_per_meter=ppm.value
                )
                print("✅ Analyse terminée !\n")

                if isinstance(summary, dict):
                    for k, v in summary.items():
                        print(f"➡ {k}: {v}")

                # Vidéo résultante
                if out_path.exists():
                    display(Video(str(out_path)))
                    display(FileLink(str(out_path), result_html_prefix="🔗 Lien vers la vidéo : "))

                # CSV résultat
                if csv_path.exists():
                    df = pd.read_csv(csv_path)
                    display(df.head())
                    display(FileLink(str(csv_path), result_html_prefix="📊 Télécharger CSV : "))

            except Exception as e:
                print(f"❌ Erreur pendant l’analyse: {e}")

    analyze_btn.on_click(on_analyze)

# ============================================
# 🔹 تشغيل التطبيق
# ============================================
show_login()


Text(value='', description="👤 Nom d'utilisateur:", layout=Layout(width='400px'), style=TextStyle(description_w…

Password(description='🔑 Mot de passe:', layout=Layout(width='400px'), style=TextStyle(description_width='150px…

Button(button_style='success', description='Connexion', style=ButtonStyle())

Button(button_style='info', description='Créer un compte', style=ButtonStyle())

Output()